In [1]:


import pygame
import random
import pandas as pd 
from copy import deepcopy

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [23]:
!pip --version

pip 20.1.1 from C:\Users\User\anaconda3\lib\site-packages\pip (python 3.8)



In [10]:
!pip install pandas

In [11]:
dict_of_offsets = {-2:[-1,0,1],
                    -1:[-2,-1,0,1,2],
                    0:[-2,-1,0,1,2],
                    1:[-2,-1,0,1,2],
                    2:[-1,0,1]}
dx = len(dict_of_offsets.keys()) // 2

dict_of_colours = {0: "black",
                   1: "forestgreen",
                   2: "red",
                   3: "blue",
                   4: "yellow"}

In [12]:
epoch = 730 # T
limit_time_illness = 21 # T1
limit_time_vaccine = 180
limit_time_immun_con = 180 # T2
probability_immun_abs = 0 # p1
probability_infect = 0.015 # p2
probability_lethal = 0.015 # p3
probability_virus_carrier = 0.1 # p4
time_found_illness = 7

In [13]:
info = {"процент людей с иммунитетом" : [],
        "процент ограничений на внешней границе": [], 
        "население": [] ,"число живых" : [], 
        "число мёртвых": [], "число заражённых" : []}
df = pd.DataFrame(data=info)
df

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых


In [14]:
info = pd.DataFrame(data = info)
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых


In [15]:
class Cell:

    def __init__(self, state = 1, immun_abs = 0, immun_con = 0,
                 vaccine = 0, time_immun_con = 0, time_illness = 0, time_vaccine = 0):
        """
        state:
        0 - мёртв
        1 - жив, здоров
        2 - жив, болен
        """
        
        self.state = state
        self.immun_abs = immun_abs
        self.immun_con = immun_con
        self.vaccine = vaccine
        self.time_immun_con = time_immun_con
        self.time_vaccine = time_vaccine
        self.time_illness = time_illness
        

In [16]:
def check_cell(current_field, x, y , H, W):
    
    # если мёртв
    if current_field[y][x].state == 0:
        return 0
    
    # если здоров
    if current_field[y][x].state == 1:
        
        # абсолютный иммунетет
        if current_field[y][x].immun_abs :
            return 1
        
        #временный иммунетет 
        if current_field[y][x].immun_con or current_field[y][x].vaccine:
            current_field[y][x].time_immun_con += 1
            if current_field[y][x].time_immun_con >= limit_time_immun_con:
                current_field[y][x].time_immun_con = 0
                current_field[y][x].immun_con = 0
                current_field[y][x].vaccine = 0
            return 1
        
        for i in range(x - dx, x + dx):
            if 0 <= i <= W - 1:
                dy = len(dict_of_offsets[i - x]) // 2
                for j in range(y - dy, y + dy):
                    # проверка, не выходит ли за границы
                    if 0 <= j <= H - 1 and current_field[j][i].state == 2:
                        if current_field[j][i].time_illness >= time_found_illness:
                            continue
                        if random.random() <= probability_infect:
                            return 2
        return 1
        
    # если болен
    if current_field[y][x].state == 2:
        if random.random() <= probability_lethal:
            return 0
        
        current_field[y][x].time_illness +=1
        
        if current_field[y][x].time_illness >= limit_time_illness:
            current_field[y][x].time_illness = 0
            current_field[y][x].immun_con = 1
            current_field[y][x].time_immun_con = 1
            return 1
        return 2

In [17]:
def gameOver(surface):
    surface.fill(pygame.Color('black'))
    font = pygame.font.SysFont("comicsansms",32)
    red = (255, 0, 0)
    green = (0, 255, 0)
    words = font.render("Вирус захватил мир!", 1, red, green)
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                
        surface.blit(words, (800,500))
        pygame.display.update()

In [18]:
def generate_left_border(field, W):
    for x in range(1, W):
        if random.random() >= perc_external_limit:
            if random.random() <= probability_virus_carrier:
                field[1][x].state = 2
        else:
            field[1][x].state = 1

In [19]:
def generate_imun(field, H, W, percent_vaccinated_people = 0, add_abs_imun = 0):
    for x in range(0, W):
        for y in range(0, H):
            if random.random() <= probability_immun_abs + add_abs_imun:
                field[y][x].immun_abs = 1
                
    for x in range(0, W):
        for y in range(0, H):
            if field[y][x].immun_abs != 1 and random.random() <= percent_vaccinated_people:
                field[y][x].vaccine = 1
    
    return field

In [20]:
def create_info(number, field, H, W,add_abs_imun):
    
    new_row = [add_abs_imun, perc_external_limit, H*W, 0, 0, 0]
    for x in range(0, W):
        for y in range(0, H):
            if field[y][x].state == 1:
                new_row[3] += 1
            if field[y][x].state == 0:
                new_row[4] += 1
            if field[y][x].state == 2:
                new_row[5] += 1
    df.loc[number] = new_row 
    return True

In [21]:
def there_is_leak(add_abs_imun = 0, number = 0, 
                  percent_vaccinated_people = 0.03, 
                  WIDTH = 2000, HEIGHT = 2000, show = False, info = False):
    TILE = 40
    if show:
        WIDTH = 1920
        HEIGHT = 1020
        TILE = 20
        
    RES = WIDTH, HEIGHT     
    W, H = WIDTH // TILE, HEIGHT // TILE
    FPS = 1
    
    if show:
        pygame.init()
        surface = pygame.display.set_mode(RES)
        clock = pygame.time.Clock()


    
    next_field = [[Cell(1) for i in range(W)] for j in range(H)]
    current_field =  [[Cell(1) for i in range(W)] for j in range(H)]
    

    iter_count = 1
    generate_imun(next_field, H, W, percent_vaccinated_people)
    while iter_count <= epoch:
        
        if show:
            surface.fill(pygame.Color('black'))
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
            
            [pygame.draw.line(surface, pygame.Color('darkslategray'), (x, 0), (x, HEIGHT)) 
                                                             for x in range(0, WIDTH, TILE)]
            [pygame.draw.line(surface, pygame.Color('darkslategray'), (0, y), (WIDTH, y)) 
                                                             for y in range(0, HEIGHT, TILE)]
         #draw surface
        current_field = deepcopy(next_field)
        generate_left_border(current_field, W)
        for x in range(1, W - 1):
            for y in range(1, H - 1):
                if show:
                    if current_field[y][x].state:
                        colour = dict_of_colours[current_field[y][x].state]
                        if current_field[y][x].immun_con or current_field[y][x].vaccine:
                            colour = dict_of_colours[3]
                        if current_field[y][x].immun_abs:
                            colour = dict_of_colours[4]
                        pygame.draw.rect(surface, pygame.Color(colour), 
                                         (x * TILE + 2, y * TILE + 2, TILE - 2, TILE - 2))
                    
                current_field[y][x].state = check_cell(current_field, x, y, H, W)
        next_field = deepcopy(current_field)
        
        
        for x in range(1, W - 1):
            if current_field[H-2][x].state == 2:
                if show:
                    gameOver(surface)
                if info:
                    create_info(number, next_field, H, W, add_abs_imun)
                return False
    
        if show:
            pygame.display.flip()
            clock.tick(FPS)
        
        iter_count +=1
    if info:
        create_info(number, next_field, H, W, add_abs_imun)
    return True
        

        


In [22]:
#there_is_leak(percent_vaccinated_people = 0.01, show = True)

In [23]:
def calculate_critical_value(number = 0, eps = 0.001, info = True):
    start, current, end = 0, 0.5, 1
    i = 0 
    while abs(start - end) >= eps:
        
        i += 1
        if not(there_is_leak(number = number,
                             add_abs_imun = current, info = info)):
            current, start = (current + end) / 2, current
        else:
            current, end = (start + current) / 2, current
            
    return current

In [ ]:
#print(calculate_critical_value())

In [6]:
def calculate_average_critical_value(repeats = 10, eps = 0.001):
    critical_value = sum([calculate_critical_value(number = i) 
                          for i in range(repeats)]) / repeats
    return critical_value


In [41]:
#Начало экспериментов
#
#
perc_external_limit = 0
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.007227,0.0,2500.0,2307.2,153.9,38.9


In [44]:
perc_external_limit = 0.2
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.007227,0.0,2500.0,2307.2,153.9,38.9
1,0.038867,0.2,2500.0,2339.3,137.6,23.1
2,0.018164,0.2,2500.0,2334.7,149.5,15.8


In [45]:
perc_external_limit = 0.3
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.007227,0.0,2500.0,2307.2,153.9,38.9
1,0.038867,0.2,2500.0,2339.3,137.6,23.1
2,0.018164,0.2,2500.0,2334.7,149.5,15.8
3,0.051562,0.3,2500.0,2339.0,141.6,19.4


In [46]:
perc_external_limit = 0.4
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.007227,0.0,2500.0,2307.2,153.9,38.9
1,0.038867,0.2,2500.0,2339.3,137.6,23.1
2,0.018164,0.2,2500.0,2334.7,149.5,15.8
3,0.051562,0.3,2500.0,2339.0,141.6,19.4
4,0.008789,0.4,2500.0,2369.8,120.2,10.0


In [47]:
perc_external_limit = 0.5
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.007227,0.0,2500.0,2307.2,153.9,38.9
1,0.038867,0.2,2500.0,2339.3,137.6,23.1
2,0.018164,0.2,2500.0,2334.7,149.5,15.8
3,0.051562,0.3,2500.0,2339.0,141.6,19.4
4,0.008789,0.4,2500.0,2369.8,120.2,10.0
5,0.103320,0.5,2500.0,2384.8,105.2,10.0


In [25]:
perc_external_limit = 0.6
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.023438,0.6,2500.0,2398.4,93.7,7.9


In [26]:
perc_external_limit = 0.7
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.023438,0.6,2500.0,2398.4,93.7,7.9
1,0.040039,0.7,2500.0,2417.6,75.5,6.9


In [27]:
perc_external_limit = 0.8
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.023438,0.6,2500.0,2398.4,93.7,7.9
1,0.040039,0.7,2500.0,2417.6,75.5,6.9
2,0.000977,0.8,2500.0,2445.1,47.8,7.1


In [28]:
perc_external_limit = 0.9
calculate_average_critical_value()
info.loc[len(info)] = (df.mean()).tolist()
info

,процент людей с иммунитетом,процент ограничений на внешней границе,население,число живых,число мёртвых,число заражённых
0,0.023438,0.6,2500.0,2398.4,93.7,7.9
1,0.040039,0.7,2500.0,2417.6,75.5,6.9
2,0.000977,0.8,2500.0,2445.1,47.8,7.1
3,0.029883,0.9,2500.0,2439.3,59.6,1.1
